In [1]:
from dataset import dataloader, Preprocess
batch_size = 128
train_class_weights, train_dataloader = dataloader(r"C:\Users\CVML_5\Desktop\ziyang\hw\data\train",preprocess = Preprocess["train"], batch_size=batch_size, shuffle=True)
_, valid_dataloader = dataloader(r"C:\Users\CVML_5\Desktop\ziyang\hw\data\val",preprocess = Preprocess["val"], batch_size=1, shuffle=False)
test_image_path = r"C:\Users\CVML_5\Desktop\ziyang\hw\data\test"

In [2]:
import torch
import torchvision.models as models
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_classes = 100
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(p = 0.2),
    torch.nn.Linear(in_features=2048, out_features=num_classes)
)
total_params = sum(p.numel() for p in model.parameters())
print(f"#parameter: {total_params}")

#parameter: 23712932


In [3]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss(weight=train_class_weights.to(device))
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [32]:
num_epochs = 200
training_curve = [[]] * 4
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    loss = running_loss/len(train_dataloader)
    accuracy = 100 * correct / len(train_dataloader) / batch_size
    training_curve[0].append(loss)
    training_curve[1].append(accuracy)
    print(f"Epoch {epoch+1}, Training loss: {loss:.6f}, Training accuracy: {accuracy:.4f}")

    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in valid_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            running_loss += loss.item()

    loss = running_loss/len(valid_dataloader)
    accuracy = 100 * correct / len(valid_dataloader)
    training_curve[2].append(loss)
    training_curve[3].append(accuracy)
    print(f"Validation loss: {loss:.6f}, Validation accuracy: {accuracy:.4f}")

Epoch 1, Training loss: 0.008042, Training accuracy: 99.7010
Validation loss: 0.768679, Validation accuracy: 86.0000


KeyboardInterrupt: 

In [43]:
torch.save(model.state_dict(), "model.pth") 

In [ ]:
import pandas as pd
import numpy as np
data = np.array([(np.array(training_curve[0])[i::4]) for i in range(4)]).T
df = pd.DataFrame(data, columns=['training_loss', 'training_acc', 'val_loss', 'val_acc'])
df.to_csv('curve.csv', index=False)

In [40]:
import os
from torchvision.io import read_image
from torchvision import transforms
image_list = os.listdir(test_image_path)
preprocess = Preprocess["train"]
name_list = []
label_list = []
for image in image_list:
    model.eval()
    img_pth = os.path.join(test_image_path, image)
    name = image[:-4]
    img = read_image(img_pth)[:3, :, :]
    if img.shape[0] == 1:
        img = img.repeat(3, 1, 1)
    img = transforms.functional.to_pil_image(img)
    img = preprocess(img).unsqueeze(0)

    inputs = img.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    name_list.append(name)
    label_list.append(predicted.item())

In [42]:
data = np.array([name_list, label_list]).T
df = pd.DataFrame(data, columns=['image_name', 'pred_label'])
df.to_csv('prediction.csv', index=False)